# Lab 9
* Diego Crespo 19541
* Javier Hernandez 19202

## Parte 1 - Filtrado y preprocesamiento

1. Cargar librerías y archivos a utilizar

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import random
import json
from flare.data_science.features import domain_tld_extract


2. Cargar los datos

In [4]:
DT = [ json.loads(line) 
            for line in open('data.json', 'r', encoding='utf-8') ]

Tráfico inicial

In [5]:
print(len(DT))

746909


In [6]:
DT[0:2]

[{'timestamp': '2017-07-22T17:33:15.784100-0500',
  'flow_id': 283570730437312,
  'pcap_cnt': 18519,
  'event_type': 'alert',
  'vlan': 130,
  'src_ip': '192.168.203.200',
  'src_port': 2328,
  'dest_ip': '192.181.145.109',
  'dest_port': 445,
  'proto': 'TCP',
  'alert': {'action': 'allowed',
   'gid': 1,
   'signature_id': 2001569,
   'rev': 15,
   'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection',
   'category': 'Misc activity',
   'severity': 3}},
 {'timestamp': '2017-07-22T17:33:16.571273-0500',
  'flow_id': 1519048138171115,
  'pcap_cnt': 21531,
  'event_type': 'ssh',
  'vlan': 140,
  'src_ip': '192.168.204.60',
  'src_port': 62990,
  'dest_ip': '192.168.26.254',
  'dest_port': 22,
  'proto': 'TCP',
  'ssh': {'client': {'proto_version': '2.0',
    'software_version': 'OpenSSH_5.0'},
   'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}]

3. Como solo buscamos dominios DGA, solamente nos interesan los registros DNS. Entonces se cargan únicamente aquellos registros en cuya llave se encuentra “DNS”.

In [7]:
dns_DT = len([x for x in data if 'dns' in x])

4. Muestre la nueva cantidad de registros filtrados. Deben ser 21484.

In [8]:
print(dns_DT)

15749


5. Mostrar la información de 2 registros cualesquiera

In [9]:
dns_DT = [x for x in DT if 'dns' in x]
random.sample(dns_DT, 2)

[{'timestamp': '2017-07-22T18:29:17.949469-0500',
  'flow_id': 645155657710563,
  'pcap_cnt': 505770,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.205.188',
  'dest_port': 34880,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 49092,
   'rcode': 'NXDOMAIN',
   'rrname': 'mirrors.tummy.com'}},
 {'timestamp': '2017-07-22T19:10:22.084580-0500',
  'flow_id': 876729718426212,
  'pcap_cnt': 2638244,
  'event_type': 'dns',
  'vlan': 140,
  'src_ip': '2001:0dbb:0c18:0014:34fa:4d67:66fe:82e9',
  'src_port': 52273,
  'dest_ip': '0000:0000:0000:0000:0000:ffff:c0a8:19b1',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 61944,
   'rrname': 'VERSION.BIND',
   'rrtype': 'TXT',
   'tx_id': 0}}]

6. Como la data consiste en json anidados, se utiliza la característica json_normalize para normalizar la información y asignarla en un dataframe.

In [10]:
DTnorm = pd.json_normalize(DT)

7. Como estamos buscando dominios DGA, se filtran los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió bajar la cantidad a 2849 registros.

In [11]:
arec = DTnorm[DTnorm['dns.rrtype']==('A')][['dns.rrname','dns.rrtype']]
arec

,dns.rrname,dns.rrtype
2,api.wunderground.com,A
38,stork79.dropbox.com,A
39,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A
43,api.wunderground.com,A
44,safebrowsing.clients.google.com.home,A
...,...,...
714913,secure.informaction.com.hsd1.pa.comcast.net,A
715127,192.168.22.201:.stayonline.net,A
715804,wpad.aol.aoltw.net,A
715910,client-software.real.com,A


In [12]:
print("Numero de registros A: ", len(arec))

Numero de registros A:  2849


8. Filtrar los dominios únicos y asignarlos a un dataframe. Debe obtener 177 registros únicos.

In [13]:
uniquer = arec['dns.rrname'].unique()
print("Numero de registros A unicos: ", len(uniquer))

Numero de registros A unicos:  177


In [14]:
uniquer.tolist()

['api.wunderground.com',
 'stork79.dropbox.com',
 'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
 'safebrowsing.clients.google.com.home',
 'fxfeeds.mozilla.com',
 'www.metasploit.com',
 'aolmtcmxm03.office.aol.com',
 'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
 'aolmtcmxm02.office.aol.com',
 'hpca-tier2.office.aol.com',
 'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
 'aolmtcmxm04.office.aol.com',
 'safebrowsing.clients.google.com',
 'wpad.home',
 'safebrowsing.clients.google.com.stayonline.net',
 'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
 'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
 'AOLDTCMA04.office.aol.com',
 '192.168.22.110phpmyadmin',
 'secure.informaction.com',
 'secure.informaction.com.localdomain',
 'safebrowsing.clients.google.com.localdomain',
 'ueip.vmware.com',
 'en-us.fxfeeds.mozilla.com',
 '192.168.22.110phpmyadmin.localdomain',
 'time.windows.com',
 'softwareupdate.vmware.com',
 'proxim.ntkrnlpa.info',
 'portswigger.net',
 'www.offensive-security.com',
 'www.of

9. Del dataframe de dominios únicos de tipo A, obtener el TLD (top level domain) utilizando Flare
como una columna nueva llamada domain_tld, y eliminar todas las demás columnas.

In [15]:
df = pd.DataFrame(uniquer, columns=['domain'])

In [16]:
domain = []
dnsDT = df.reset_index()
for col, row in dnsDT.iterrows():
    domain.append(domain_extract(row['domain']))

dnsDT['domain_tld'] = domain

In [17]:
dnsDT = dnsDT[['domain_tld']]
dnsDT.head()

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,safebrowsing.clients.google.com.home
4,mozilla.com


10. Buscar y eliminar un dominio que es un carácter vacío ‘’. 

También elimine los siguientes
dominios:
a. 192.168.22.110phpmyadmin

b. 192.168.22.110phpmyadmin.localdomain

c. "192.168.206.56"

d. 192.168.26-27.0

e. 192.168.21.1201

f. 1922.168.22.254

g. 1922.168.22.254.home

h. 192.168.21-28.0.home

i. 192.168.22.201:

j. 192.168.22.201:.stayonline.net

In [18]:
remove = ['', ' ', '192.168.22.110phpmyadmin', '192.168.22.110phpmyadmin.localdomain', '"192.168.206.56"', '192.168.26-27.0', '192.168.21.1201', '1922.168.22.254', '1922.168.22.254.home', '192.168.21-28.0.home', '192.168.22.201:', '192.168.22.201:.stayonline.net']
clean = dnsDT[~dnsDT['domain_tld'].isin(remove)]

In [19]:
clean = clean.drop_duplicates(subset=['domain_tld'])

In [20]:
clean

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,safebrowsing.clients.google.com.home
4,mozilla.com
...,...
166,192.168.21-28.0
167,ejfodfmfxlkgifuf.xyz
172,ntp.ubuntu.com.localdomain
174,verisign.com


11. Obtener los TLD únicos. Debe obtener 102 registros.

In [21]:
print("TLD unicos: ", len(clean))

TLD unicos:  102


## Parte 2 - Data Science

1. Utilizar el clasificador proporcionado, pasar como parámetro el dataframe con la columna domain_tld, y asignar el resultado a un nuevo DF.

In [22]:
import clasificador

In [23]:
new = clasificador.clasificacion(clean)

In [24]:
new

,domain_tld,isDGA
0,wunderground.com,0
1,dropbox.com,1
2,aoltw.net,1
3,safebrowsing.clients.google.com.home,0
4,mozilla.com,0
...,...,...
166,192.168.21-28.0,1
167,ejfodfmfxlkgifuf.xyz,1
172,ntp.ubuntu.com.localdomain,1
174,verisign.com,0


2. Filtrar aquellos considerados como DGA (valor 1 ) y mostrarlos. Recuerde que los modelos de ML ofrecen una predicción, pero los resultados pueden ser falsos positivos y falsos negativos, por lo que no podemos fiarnos por completo de esta clasificación y debemos seguir indagando. Ahora tenemos solo 35 dominios sospechosos.

In [66]:
df_DGA = new_df[(new['isDGA'] == 1)]

In [67]:
print("DGA: ", len(df_DGA))

DGA:  35


## Parte 3 - Dominio experto

1. Ya se cuenta con un listado de dominios reducido y considerados como sospechosos, entonces se aplica el dominio experto para encontrar los verdaderos registros malicioso. Utilizando la lista de Cisco Umbrella incluido en Flare, indicar si los dominios encontrados por el clasificador DGA de la parte 2 se encuentran en este top 1000000. Deberíamos haber reducido los registros a 12.

In [28]:
umbrella = pd.read_csv("./flare/data/umbrella/top-1m.csv")


In [32]:
umbrella

,1,netflix.com
0,2,api-global.netflix.com
1,3,prod.netflix.com
2,4,push.prod.netflix.com
3,5,google.com
4,6,microsoft.com
...,...,...
999994,999996,taipei-a1.gp.schneider-electric.com
999995,999997,taiwantoday.tw
999996,999998,takethebestpics.com
999997,999999,takiedela.ru


In [68]:
df_DGA['umbrella_match'] = df_DGA['domain_tld'].isin(umbrella['netflix.com'])

/var/folders/7v/q__s863s77v1wy7dl0jwhcd00000gn/T/ipykernel_20008/3189910155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_DGA['umbrella_match'] = df_DGA['domain_tld'].isin(df_umbrella['netflix.com'])


In [81]:
df_DGA

,domain_tld,isDGA,umbrella_match
1,dropbox.com,1,True
2,aoltw.net,1,False
5,metasploit.com,1,True
25,windows.com,1,True
27,ntkrnlpa.info,1,False
34,sql-ledger.org,1,False
35,backtrack-linux.org,1,True
48,phpmyadmin.net,1,True
51,microsoft.com,1,True
60,postgresql.org,1,True


In [112]:
new_df_DGA = df_DGA[(df_DGA['umbrella_match'] == True)]

In [108]:
print("DGA: ", len(new_df_DGA))

DGA:  25


2. Si son considerados por el clasificador como DGA, y no están en el top de Cisco Umbrella, es muy probable que sean maliciosos. Para confirmar esto podemos buscar la fecha de creación del dominio. Crear una función que en base al dominio, devuelva la fecha de creación de este (utilizar la librería whois para esto).

In [49]:
import whois

In [98]:
whois.whois(new_df_DGA['domain_tld'][1]).creation_date

datetime.datetime(1995, 6, 28, 4, 0)

In [128]:

for i in new_df_DGA['domain_tld']:
    new_df_DGA['creation_date'] = pd.Series(whois.whois(i).creation_date)


# new_df_DGA.drop('creation_date', axis=1, inplace=True)



/var/folders/7v/q__s863s77v1wy7dl0jwhcd00000gn/T/ipykernel_20008/2004958218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_DGA['creation_date'] = pd.Series(whois.whois(i).creation_date)
/var/folders/7v/q__s863s77v1wy7dl0jwhcd00000gn/T/ipykernel_20008/2004958218.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df_DGA['creation_date'] = pd.Series(whois.whois(i).creation_date)
/var/folders/7v/q__s863s77v1wy7dl0jwhcd00000gn/T/ipykernel_20008/2004958218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [129]:
new_df_DGA

,domain_tld,isDGA,umbrella_match,creation_date
1,dropbox.com,1,True,1996-12-24
5,metasploit.com,1,True,NaT
25,windows.com,1,True,NaT
35,backtrack-linux.org,1,True,NaT
48,phpmyadmin.net,1,True,NaT
51,microsoft.com,1,True,NaT
60,postgresql.org,1,True,NaT
62,freepbx.org,1,True,NaT
78,flickr.com,1,True,NaT
83,facebook.com,1,True,NaT


3. Mostrar la fecha de creación para cada uno de los dominios clasificados como DGA. ¿Cuáles son los dominios que podemos confirmar como sospechosos? (Deben ser cinco dominios)

4. Los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de
caracteres, no palabras. Indicar qué dominios sospechosos tienen este patrón y que pueden confirmarse como dominios DGA (deben ser dos).